In [1]:
import os
import openai
import toml
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login, hf_hub_download

# Define the path to the secrets.toml file
secrets_file_path = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', '.streamlit', 'secrets.toml'))

# Load the secrets
secrets = toml.load(secrets_file_path)

# Accessing the Hugging Face token and OpenAI token from the secrets
hf_token = secrets["huggingface_token"]
openai_token = secrets["openai_token"]

# Log in to Hugging Face
login(hf_token, add_to_git_credential=True)

/home/EUR/moabd/.cache/pypoetry/virtualenvs/gllm-Z7QQw8HW-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/EUR/moabd/.cache/huggingface/token
Login successful


In [2]:
# load the base model
config = PeftConfig.from_pretrained("ArneKreuz/starcoderbase-finetuned-thestack", token=hf_token)
base_model = AutoModelForCausalLM.from_pretrained("bigcode/starcoderbase-3b", token=hf_token)

/home/EUR/moabd/.cache/pypoetry/virtualenvs/gllm-Z7QQw8HW-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.97it/s]


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoderbase-3b")

# If the tokenizer does not have a padding token, set it to be the same as eos_token
if tokenizer.pad_token is None: 
    tokenizer.pad_token = tokenizer.eos_token

In [4]:
# Load the fine tuned model
model = PeftModel.from_pretrained(base_model, "ArneKreuz/starcoderbase-finetuned-thestack", token=hf_token, force_download=True)

In [9]:
instructions = """
Generate G-code with the following instructions: 
Machine: Siemens 840D CNC Mill
Operation: Milling
Material: Steel
Workpiece Dimensions: 100mm x 100mm x 20mm
Tool Selection:
  - Milling Tool: Carbide end mill, 10mm diameter, 4-flute
Toolpath Strategy: The sequence includes moving to the start position, cutting a square shape in the material, and then returning to the home position to end the program.
Machining Parameters:
  - Cutting Speed: 500 m/min
  - Feed Rate: 200 mm/min
  - Depth of Cut: 5 mm
  - Stepover: 50%
Safety Considerations:
  - Rapid Retract Height: 10 mm
  - Tool Clearance: 2 mm
Your answer should have only the G-code without explanations.
"""

In [8]:
# Encode the instructions and generate the G-code
input_ids = tokenizer.encode(instructions, return_tensors='pt')
attention_mask = input_ids.ne(tokenizer.pad_token_id).int()
output = base_model.generate(input_ids, attention_mask=attention_mask, max_length=2000, temperature=0.7)

# Decode the output to get the G-code
g_code = tokenizer.decode(output[0])

with open('output.txt', 'w') as f:
    f.write(g_code)

/home/EUR/moabd/.cache/pypoetry/virtualenvs/gllm-Z7QQw8HW-py3.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
